## Python notebook 
V nasledujúcom notebooku sú popísané Python metódy použité pre analýzu dát. Obsah notebooku je nasledujúci:
1. Import potrebných knižníc
2. Spracovanie dát z YOLO výstupu
3. Predikcia modelom pre klasifikáciu disperzií
4. Vytvorenie konečnej výstupnej tabuľky

# 1. Import potrebných knižníc

In [1]:
!pip install -r requirements.txt

     |████████████████████████████████| 60.5 MB 60.0 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 11.2 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 55.0 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 73.9 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 63.0 MB/s eta 0:00:01
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61086 sha256=d7ad3d3b90ddfff60bdabc4d38d4022945fa76410dcc6f7f3337bc3dce9ecb74
  Stored in directory: /home/jovyan/.cache/pip/wheels/b1/1a/8f/a4c34be976825a2f7948d0fa40907598d69834f8ab5889de11
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=53440820352bbe9d6e224745d7f772fdb13a5cafc7f3560ba47e2ac620c6be15
  Stored in directory: /home/jovyan/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built PyPDF2 future
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.4.0
    

In [2]:
import csv
import os
import cv2
import numpy as np
import operator
from skimage.io import imread
from PIL import Image, ImageDraw
import pickle
import random
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, show

import tensorflow
from tensorflow import keras
from tensorflow.keras.models import load_model

In [3]:
#funkcia na odstránenie udalostí, ktoré YOLO detegovalo ako tweek a aj ako sferik. Predpoklad vymazania udalosti je
#detekovanie v rovnakej milisekunde
#odstraňuje sa udalosť typu sferik v tabuľky
def clear_duplicate(df):
    mil=[]
    tweek=[]
    for i in df['milisecond']:
        mil.append(i)

    for i in df['tweek']:
        tweek.append(i)

    for index, elem in enumerate(mil):
        if (index+1 <= len(mil) and index - 1 >= 0):
            prev_el = str(mil[index-1])
            curr_el = str(elem)
            if prev_el == curr_el:
                if tweek[index]>tweek[index-1]:
                    df=df.drop(df.index[index-1])
                    mil.pop(index-1)
                    tweek.pop(index-1)
                else:
                    df=df.drop(df.index[index])
                    mil.pop(index)
                    tweek.pop(index)
    return df

# 2. Spracovanie dát z YOLO výstupu
Z YOLO detekcie extrahujeme nájdené udalosti podľa typu (tweek, sferik) a udalosti typu tweek ďalej klasifikujeme podľa počtu disperzií.

In [4]:
#image_path - spracované surové dáta, na ktorých bola použitá YOLO metóda
#croppedPath - zložka, kam sa majú uložiť extrahované sferiky
#rootPath - zložka s označením tried YOLO metódou (label dát)
#imgNR - poradové číslo udalosti
imagePath= r'/home/jovyan/data/lightning/Samuel/git/tweeks_detection/files/2016_Data/2016_Img'
croppedPath = r'/home/jovyan/data/lightning/Samuel/git/tweeks_detection/files/2016_Data/2016_Cropped/2016_Cropped_S'
rootPath = r'/home/jovyan/data/lightning/Samuel/git/tweeks_detection/yolov5/runs/detect/2016/labels'
path = ""
imageNR=1

Udalosti typu sferik

In [ ]:
#extrahuje sferiky a vytvorí čiastkovú výstupnú tab. sferikov output_S.csv
with open(os.path.join("output_S_2016.csv"), "w") as outputCsv:    
    fieldnamesOutput = ['image','event', 'date', 'second', 'milisecond', 'tweek', 'f_min<2kHz', 'tweek_sign']
    tableWriter = csv.DictWriter(outputCsv, fieldnames=fieldnamesOutput, delimiter=',')
    tableWriter.writeheader()
    
    for root, dirs, files in os.walk(rootPath):
        for file in files:
            with open(os.path.join(root, file), "r") as csv_file:
                reader = csv.reader(csv_file, delimiter=' ')
                sortedlist = sorted(reader, key=operator.itemgetter(1))
            with open(os.path.join(root, file), "w") as csv_file1:
                writer = csv.writer(csv_file1, delimiter=' ')
                writer.writerows(sortedlist)

            with open(os.path.join(root, file), "r") as coordFile:

                fieldnamesCoords = ['label','x', 'y', 'width', 'height', 'conf']
                eventNR=1
                coordReader = csv.DictReader(coordFile, fieldnames=fieldnamesCoords, delimiter=' ')
                try:
                    img = Image.open(os.path.join(imagePath, file.split(".")[0]+".jpg"))
                except:
                    break
                w, h = img.size
                widthms=w/1000

                for row in coordReader:
                    if row['label'] == "0":
                        if row['conf'] > "0.01":
                            widthCoord=round(float(row['width'])*w)
                            xCoord=round((float(row['x'])*w)-(float(widthCoord)/2))
                            im = img.crop((xCoord, 0, xCoord+widthCoord, 162)).save(os.path.join(croppedPath, file.split(".")[0]+"-S%d.jpg"%eventNR))
                            image = file.split(".")[0]+"-S%d.jpg"%eventNR
                            im2 = img.crop((xCoord, 0, xCoord+widthCoord, 162)).save(os.path.join(root, "1.jpg"))
                        
                            red = imread(os.path.join(root, "1.jpg"))[:, :, 0]
                            os.remove(os.path.join(root, "1.jpg"))

                            red=np.delete(red,len(red)-1,0)
                            red=np.delete(red,len(red)-1,0)

                            transposered=np.transpose(red)

                            middle=0
                            indexOfMid=0
                            currentIndex=1
                            for i in transposered:
                                if sum(i) > middle:
                                    middle=sum(i)
                                    indexOfMid=currentIndex
                                currentIndex+=1

                            milsec=round((indexOfMid+xCoord)/widthms)
                        
                            j=0
                            celyobr=0
                            konec=0
                            while(j<160):
                                celyobr+=sum(red[j])
                                if j>138:
                                    konec+=sum(red[j])

                                j+=1

                            if (float((konec*100)/celyobr))>9.5:
                                vysledok=1
                            else: vysledok=0
                            
                        
                            tableWriter.writerow({
                                'image':image, 
                                'event':eventNR, 
                                'date':file[4:19], 
                                'second':file[file.find("s")+1:file.find(".txt")], 
                                'milisecond':milsec, 
                                'tweek':row['label'], 
                                'f_min<2kHz':vysledok,
                                'tweek_sign':'0'})
                            eventNR+=1
                            #print(row)
                        imageNR+=1

In [6]:
#image_path - spracované surové dáta, na ktorých bola použitá YOLO metóda
#croppedPath - zložka, kam sa majú uložiť extrahované tweeky
#rootPath - zložka s označením tried YOLO metódou (label dát)
#imgNR - poradove číslo udalosti
#imagePath= r'/home/jovyan/data/lightning/Samuel/git/tweeks_detection/files/pred_img'
croppedPath = r'/home/jovyan/data/lightning/Samuel/git/tweeks_detection/files/2016_Data/2016_Cropped/2016_Cropped_T'
#rootPath = r'/home/jovyan/data/lightning/Samuel/git/tweeks_detection/yolov5/runs/detect/exp/labels'
path = ""
imageNR=1

Udalosti typu tweek

In [ ]:
#extrahuje tweeky a vytvorí čiastkovú výstupnú tab. tweekov output_T.csv
with open(os.path.join("output_T_2016.csv"), "w") as outputCsv:    
    fieldnamesOutput = ['image','event', 'date', 'second', 'milisecond', 'tweek', 'f_min<2kHz']
    tableWriter = csv.DictWriter(outputCsv, fieldnames=fieldnamesOutput, delimiter=',')
    tableWriter.writeheader()
    
    for root, dirs, files in os.walk(rootPath):
        for file in files:
            with open(os.path.join(root, file), "r") as csv_file:
                reader = csv.reader(csv_file, delimiter=' ')
                sortedlist = sorted(reader, key=operator.itemgetter(1))
            with open(os.path.join(root, file), "w") as csv_file1:
                writer = csv.writer(csv_file1, delimiter=' ')
                writer.writerows(sortedlist)

            with open(os.path.join(root, file), "r") as coordFile:

                fieldnamesCoords = ['label','x', 'y', 'width', 'height', 'conf']
                eventNR=1
                coordReader = csv.DictReader(coordFile, fieldnames=fieldnamesCoords, delimiter=' ')
                try:
                    img = Image.open(os.path.join(imagePath, file.split(".")[0]+".jpg"))
                except:
                    break
                w, h = img.size
                widthms=w/1000

                for row in coordReader:
                    if row['label'] == "1":
                        widthCoord=round(float(row['width'])*w)
                        xCoord=round((float(row['x'])*w)-(float(widthCoord)/2))
                        im = img.crop((xCoord, 0, xCoord+widthCoord, 162)).save(os.path.join(croppedPath, file.split(".")[0]+"-T%d.jpg"%eventNR))
                        image = file.split(".")[0]+"-T%d.jpg"%eventNR
                        im2 = img.crop((xCoord, 0, xCoord+widthCoord, 162)).save(os.path.join(root, "1.jpg"))
                        
                        red = imread(os.path.join(root, "1.jpg"))[:, :, 0]
                        os.remove(os.path.join(root, "1.jpg"))

                        red=np.delete(red,len(red)-1,0)
                        red=np.delete(red,len(red)-1,0)

                        transposered=np.transpose(red)

                        middle=0
                        indexOfMid=0
                        currentIndex=1
                        for i in transposered:
                            if sum(i) > middle:
                                middle=sum(i)
                                indexOfMid=currentIndex
                            currentIndex+=1

                        milsec=round((indexOfMid+xCoord)/widthms)
                        
                        j=0
                        celyobr=0
                        konec=0
                        while(j<160):
                            celyobr+=sum(red[j])
                            if j>138:
                                konec+=sum(red[j])

                            j+=1

                        if (float((konec*100)/celyobr))>9.5:
                            vysledok=1
                        else: vysledok=0
                            
                        
                        tableWriter.writerow({
                            'image':image, 
                            'event':eventNR, 
                            'date':file[4:19], 
                            'second':file[file.find("s")+1:file.find(".txt")], 
                            'milisecond':milsec, 
                            'tweek':row['label'], 
                            'f_min<2kHz':vysledok})
                        eventNR+=1
                        #print(row)
                    imageNR+=1

# 3. Predikcia modelom pre klasifikáciu disperzií

Extrahované udalosti typu tweek klasifikujeme vytvoreným model podľa počtu disperzií.

In [8]:
#DIRECTORY - cesta ku extrahovaným tweekom
#model - natrénovaný model klasifikácie disperzií tweekov
DIRECTORY = r'/home/jovyan/data/lightning/Samuel/git/tweeks_detection/files/2016_Data/2016_Cropped/2016_Cropped_T'
model = load_model('model_datagen_RBG.h5')

In [ ]:
#spracuje extrahované tweeky do formátu vhodného pre klasifikáciu modelom
#na dátach predikuje počet tweek_sign (disperzií)
#predikciu spojí s čiastkovou výstupnou tab. output_T.csv, upraví číslovanie predikcií a pridá ich do výst. tab. vyst_tab.csv
data2 = []
pathvar = []
folder = DIRECTORY
for img in os.listdir(folder):
    img_path = os.path.join(folder, img)
    if(img_path == '/home/jovyan/data/lightning/Samuel/git/tweeks_detection/files/2016_Data/2016_Cropped/2016_Cropped_T/.ipynb_checkpoints'):
        continue
    #print(img_path)
    img_arr = cv2.imread(img_path)
    img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
    img_arr = cv2.resize(img_arr, (40, 150))
    pathvar.append(img)
    data2.append(img_arr)
    
X = []
for features in data2:
    X.append(features)
X = np.asarray(X, dtype=np.float32)/255
#X = np.array(X)
#X = X/255
pathvar = np.array(pathvar)

predictions = model.predict_classes(X, batch_size = 30)
#print(predictions)

df = pd.DataFrame(list(zip(pathvar, predictions)), columns =['image', 'tweek_sign'])

output = pd.read_csv('output_T_2016.csv')

final = pd.merge(output, df, on="image")
final['tweek_sign'].replace({0:1, 1:2, 2:3, 3:4}, inplace = True)
final.to_csv('vyst_tab_2016.csv', index = False)
final

# 4. Vytvorenie konečnej výstupnej tabuľky

Čiastkové výstupné tabuľky spojíme do konečnej výstupnej tabuľky s názvom result. Konečnú výstupnú tabuľku usporiadame aby udalosti išli za sebou ako boli detegované a očistíme tabuľku od duplikátov funkciou, ktorú sme definovali na začiatku notebooku.

In [10]:
#spojí čiastkové výst. tab. sferikov output_S.csv a tweekov vyst_tab.csv
df1 = pd.read_csv("output_S_2016.csv", low_memory=False)
df2 = pd.read_csv("vyst_tab_2016.csv", low_memory=False)

In [11]:
frames = [df1, df2]
result = pd.concat(frames)

In [12]:
#vytvorí nový stĺpec s názvom pôvodného spektrogramu, z ktorého udalosť pochádza
#usporiada hodnoty najprv podľa pôvodného spektrogramu 'org_img' a potom podľa milisekundy 'milisecond'
#upraví indexovanie
result['org_img'] = result['image'].str.split('-', 1).str[0]
result = result[['org_img', 'image', 'event', 'date', 'second', 'milisecond', 'tweek', 'f_min<2kHz', 'tweek_sign']]
result = result.sort_values(['org_img', 'milisecond'])
result = result.reset_index()
result = result.drop(['index'], axis = 1)

In [ ]:
#zavolá funkciu na odstránenie duplicitne zaklasifikovaných udalostí
result = clear_duplicate(result)

In [14]:
#celkovú výst. tab. uloží ako csv result.csv
result.to_csv('result_2016.csv', index=False)

Duplicitné udalosti je možné vymazať zo zložiek, v ktorých sú uložené nasledujúcim kódom. Pre potreby analýzy dát nie je tento krok potrebný. Potrebný je iba vtedy, ak by sme dáta chceli použiť aj pre potreby nekontrolovaného učenia.

In [15]:
#rootPath - cesta ku extrahovaným sferikom
#file_name - zoznam súborov, ktoré nechceme vymazať (názov img zhodný z názvom v rootPath)
rootPath = r'/home/jovyan/data/lightning/Samuel/git/tweeks_detection/files/2016_Data/2016_Cropped/2016_Cropped_S'
file_name = result['image'].tolist()

In [16]:
rootPath

'/home/jovyan/data/lightning/Samuel/git/tweeks_detection/files/2016_Data/2016_Cropped/2016_Cropped_S'

In [17]:
len(file_name)

4417696

In [ ]:
#prejde uložené obrázky a ak sa obrázok nevyskytuje v zozname file_name, vymaže udalosť zo zložky
#maže duplicitne označené udalosti YOLO metódou.
for filename in os.listdir(rootPath):
    #print(filename)
    if filename not in file_name:
        if(filename == '.ipynb_checkpoints'):
            continue
        else:
            full_file_path = os.path.join(rootPath, filename)
            os.remove(full_file_path)
            print("removed: "+filename)